<a href="https://colab.research.google.com/github/DarkLord-13/AI_Agents_For_Daily_Use/blob/main/AI_Agent_MCP_Basic_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install langchain langchain_community chromadb sentence-transformers &> /dev/null

In [8]:
# ### COLAB MODIFICATION ###
# Now, we set up the Ollama server. This involves downloading the installation
# script, running it, and then starting the server as a background process.
# This makes the 'llama3' model available for our script to use.
import os
import asyncio

Setting up Ollama model

In [3]:
!pip install ollama

In [5]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
# Start Ollama as a background process
os.environ['OLLAMA_HOST'] = '0.0.0.0'
os.environ['OLLAMA_ORIGINS'] = '*'
command = "nohup ollama serve&"
proc = await asyncio.create_subprocess_shell(
    command,
    stdout=asyncio.subprocess.PIPE,
    stderr=asyncio.subprocess.PIPE
)
print(f"Ollama server started with PID: {proc.pid}")
await asyncio.sleep(5) # Give the server a moment to start

Ollama server started with PID: 4520


In [10]:
!ollama pull llama3

In [11]:
# ==============================================================================
# Step 1: Create the Knowledge Base (The "Library")
# ==============================================================================
# This part remains the same. It's the offline "Context Ingestion" part of MCP.
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.agents import tool, AgentExecutor, create_react_agent
from langchain import hub

In [12]:
# Simple text about the Apollo 11 mission to serve as our knowledge source.
apollo_text = """
The Apollo 11 mission was the first to land humans on the Moon.
The commander was Neil Armstrong and the pilot was Buzz Aldrin.
Armstrong was the first person to step onto the lunar surface on July 20, 1969.
His first words were, 'That's one small step for a man, one giant leap for mankind.'
The crew returned to Earth on July 24, 1969. The third crew member was Michael Collins, who piloted the command module.
"""

In [13]:
# Chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.create_documents([apollo_text])

In [14]:
embeddings = OllamaEmbeddings(model="llama3", base_url="http://localhost:11434")
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

/tmp/ipython-input-14-96628743.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3", base_url="http://localhost:11434")


In [15]:
# ==============================================================================
# Step 2: Create the Tool (The Agent's "Hands")
# ==============================================================================
# This part remains the same. We define a tool that the agent can use.
@tool
def get_apollo_mission_info(query: str) -> str:
    """Searches and returns information about the Apollo 11 mission."""
    # The retriever already knows about our vectorstore and embeddings
    retrieved_docs = retriever.invoke(query)
    return "\n\n".join([doc.page_content for doc in retrieved_docs])


tools = [get_apollo_mission_info]

In [16]:
# ==============================================================================
# Step 3: Create the Agent (The "Brain")
# ==============================================================================
# This part remains the same. We pull a pre-made "ReAct" prompt.
prompt = hub.pull("hwchase17/react")
llm = Ollama(model="llama3", base_url="http://localhost:11434")

# Combine the LLM, tools, and prompt to create the agent
agent = create_react_agent(llm, tools, prompt)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/tmp/ipython-input-16-1717944706.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3", base_url="http://localhost:11434")


In [17]:
# ==============================================================================
# Step 4: Create the Agent Executor (The "Orchestrator")
# ==============================================================================
# This part remains the same.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
# ==============================================================================
# Step 5: Run the Agent and see MCP in Action!
# ==============================================================================
# This part remains the same.
user_question = "Who was the first person on the moon and what did they say?"
response = agent_executor.invoke({"input": user_question})

print("\n--- Final Answer ---")
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To find out who was the first person on the moon, we need to get information about the Apollo 11 mission.

Action: get_apollo_mission_info
Action Input: "first astronaut on the moon"The Apollo 11 mission was the first to land humans on the Moon.
The commander was Neil Armstrong and the pilot was Buzz Aldrin.
Armstrong was the first person to step onto the lunar surface on July 20, 1969.
His first words were, 'That's one small step for a man, one giant leap for mankind.'
The crew returned to Earth on July 24, 1969. The third crew member was Michael Collins, who piloted the command module.Thought: Now that we have information about the Apollo 11 mission, let's extract the answer to our original question.

Action: None (just analyzing the observation)
Action Input: NoneNone (just analyzing the observation) is not a valid tool, try one of [get_apollo_mission_info].Let's proceed!

Thought: Since we already have the information about the Apoll